In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import numpy as np
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

In [2]:
bbc_text = pd.read_csv('bbc-text.txt')
review=bbc_text.rename(columns = {'text': 'News_Headline'}, inplace = False)
review.category = bbc_text.category.map({'tech':0, 'business':1, 'sport':2, 'entertainment':3, 'politics':4})

In [3]:
X = review.News_Headline
y = review.category
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=1)

In [4]:
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

C:\Users\riyag\AppData\Local\Temp\ipykernel_12680\2176491414.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [5]:
# Define a function to create word vectors for a given text using GloVe embeddings
def get_word_vectors(text):
    words = text.split()
    word_vectors_list = [glove_model[word] if word in glove_model else np.zeros(glove_model.vector_size) for word in words]
    return np.mean(word_vectors_list, axis=0)

In [6]:
# Transform the text data to word vectors
X_train_transformed = [get_word_vectors(text) for text in X_train]
X_test_transformed = [get_word_vectors(text) for text in X_test]

In [7]:
# Train the model
classifier = LinearSVC()
classifier.fit(X_train_transformed, y_train)

LinearSVC()

In [8]:
from sklearn.metrics import classification_report

# Predict the labels for the test data using the trained model
y_pred = classifier.predict(X_test_transformed)

# Generate the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       155
           1       0.98      0.97      0.97       210
           2       1.00      0.99      1.00       197
           3       0.99      0.99      0.99       159
           4       0.94      0.98      0.96       169

    accuracy                           0.98       890
   macro avg       0.98      0.98      0.98       890
weighted avg       0.98      0.98      0.98       890



In [9]:
review1 = ['world bank']

# Transform the review text to word vectors
review1_transformed = get_word_vectors(review1[0])

# Reshape the transformed review to match the expected input shape
review1_transformed = review1_transformed.reshape(1, -1)

# Predict the sentiment label for the review
prediction = classifier.predict(review1_transformed)[0]

# Map the predicted label to 'NEGATIVE' or 'POSITIVE'
if prediction==0:
    k='tech'
elif prediction==1:
    k='business'
elif prediction==2:
    k='sport'
elif prediction==3:
    k='entertainment'
elif prediction==4:
    k='politics'

# Print the headline and the predicted sentiment
print('Headline:', review1)
print('prediction:', k)


Headline: ['world bank']
prediction: business
